In [104]:
from sklearn.feature_extraction.text import CountVectorizer
import nltk
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn import metrics

In [105]:
stop_words = set(nltk.corpus.stopwords.words("english"))

with open('dems.txt', 'r',encoding='UTF8') as file:
    dem_text = [line.strip('\n') for line in file]

with open('gop.txt', 'r',encoding='UTF8') as file:
    gop_text = [line.strip('\n') for line in file]

vectorizer = CountVectorizer(input=dem_text + gop_text,
                             stop_words=stop_words,
                             max_features=1000)
dem_bow = vectorizer.fit_transform(dem_text)
gop_bow = vectorizer.fit_transform(gop_text)

In [106]:

col=vectorizer.get_feature_names()
df_D=pd.DataFrame(data=dem_bow.toarray(),columns=col)
df_D['class']='D'

In [107]:
import pandas as pd
df_R=pd.DataFrame(data=gop_bow.toarray(),columns=col)
df_R['class']='R'

In [108]:
df=df_D.append(df_R, ignore_index=True)
df_summ=df.describe().transpose()
#df_summ.where(df_summ['max']>=2,inplace=True)
df_summ.to_csv('summ.csv')

In [109]:
corr_matrix=df.corr().abs()
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))
to_drop = [column for column in upper.columns if any(upper[column] < 0.5)]

In [110]:
to_drop

['100',
 '15',
 '19',
 '500',
 '5m',
 '7cmg0flarj',
 '7pm',
 '87lfpyu3pq',
 '88022',
 '8fiqwe2sut',
 '92alqqhuvd',
 '95',
 '96',
 '9jxdlvuffc',
 '9xedjy0eer',
 '9zvzvymzhb',
 'abc',
 'ability',
 'able',
 'absolutely',
 'access',
 'according',
 'accountable',
 'across',
 'act',
 'acti',
 'action',
 'actions',
 'activated',
 'activating',
 'activists',
 'acts',
 'actually',
 'ad',
 'add',
 'adding',
 'adjust',
 'admin',
 'administered',
 'administration',
 'adopted',
 'adults',
 'advance',
 'advice',
 'advocates',
 'adzgxne4g7',
 'affected',
 'afternoon',
 'agencies',
 'agenda',
 'agendas',
 'aggressive',
 'agree',
 'agreed',
 'ahead',
 'aid',
 'aisle',
 'ajxkab7xn6',
 'alone',
 'also',
 'always',
 'amazing',
 'america',
 'american',
 'americans',
 'amp',
 'andrew',
 'announced',
 'another',
 'answer',
 'anyone',
 'apart',
 'approach',
 'approve',
 'april',
 'arizona',
 'around',
 'ask',
 'assistance',
 'available',
 'back',
 'badly',
 'banking',
 'beginning',
 'behind',
 'beyond',
 'bid

In [103]:
df1=df[to_drop]
corrmat = df1.corr()
top_corr_features = corrmat.index
plt.figure(figsize=(20,20))
#plot heat map
g=sns.heatmap(df1[top_corr_features].corr(),annot=True,cmap="RdYlGn")

KeyboardInterrupt: 

Error in callback <function install_repl_displayhook.<locals>.post_execute at 0x000002055E583798> (for post_execute):


KeyboardInterrupt: 

Error in callback <function flush_figures at 0x000002055E65DB88> (for post_execute):


KeyboardInterrupt: 

In [111]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
X = df.iloc[:,0:119]  #independent columns
y = df.iloc[:,-1]    #target column i.e price range
#apply SelectKBest class to extract top 10 best features
bestfeatures = SelectKBest(score_func=chi2, k=10)
fit = bestfeatures.fit(X,y)
dfscores = pd.DataFrame(fit.scores_)
dfcolumns = pd.DataFrame(X.columns)
#concat two dataframes for better visualization 
featureScores = pd.concat([dfcolumns,dfscores],axis=1)
featureScores.columns = ['Specs','Score']  #naming the dataframe columns
print(featureScores.nlargest(10,'Score'))  #print 10 best features

          Specs      Score
51      agendas  28.846154
9         88022  18.512821
10   8fiqwe2sut  18.512821
11   92alqqhuvd  18.512821
12           95  18.512821
106   candidate  11.700855
70       answer   8.307692
13           96   6.564103
80    available   2.307692
66          amp   1.153846


In [112]:
X = df.loc[:, df.columns != 'class']  # Features
y = df['class'] 

In [118]:
#PCA
pca=PCA()
principalComponents=pca.fit_transform(X)
principalDf=pd.DataFrame(data=principalComponents)
principalDf
X = principalDf # Features
y = df['class'] 

In [119]:
explained_variance = pca.explained_variance_ratio_
explained_variance

array([5.09168648e-02, 3.24412202e-02, 2.72781766e-02, 1.81200436e-02,
       1.60002029e-02, 1.50909426e-02, 1.44563488e-02, 1.37945955e-02,
       1.24643127e-02, 1.22381382e-02, 1.19496281e-02, 1.11964134e-02,
       9.86275312e-03, 9.68406822e-03, 9.44666818e-03, 9.22517482e-03,
       9.11371839e-03, 8.35644227e-03, 8.00334545e-03, 7.89117647e-03,
       7.80029776e-03, 7.54722953e-03, 7.45792868e-03, 7.26223635e-03,
       7.03467544e-03, 6.99469211e-03, 6.72891211e-03, 6.72756153e-03,
       6.52611285e-03, 6.41054059e-03, 6.29116474e-03, 6.15180542e-03,
       6.09216446e-03, 5.90898509e-03, 5.85625228e-03, 5.70928144e-03,
       5.66229777e-03, 5.55269525e-03, 5.50432105e-03, 5.46221451e-03,
       5.38342885e-03, 5.26639572e-03, 5.26017411e-03, 5.20193308e-03,
       5.10528695e-03, 5.05496858e-03, 5.05057085e-03, 4.94218138e-03,
       4.90183465e-03, 4.84579623e-03, 4.79919422e-03, 4.72257079e-03,
       4.65830946e-03, 4.54818268e-03, 4.52435187e-03, 4.47872731e-03,
      

In [120]:
# 70% training and 30% test

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [121]:
#Create a Gaussian Classifier
gnb = GaussianNB()

#Train the model using the training sets
gnb.fit(X_train, y_train)

#Predict the response for test dataset
y_pred = gnb.predict(X_test)

# Model Accuracy, how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.6916666666666667
